In [37]:
import os
import sys
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()

strong_llm = ChatOpenAI(model="gpt-4-0613", temperature=0.3)
fast_llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0.3)

with open(file="prompts/generators/INITIAL_HISTORY_TCC_GENERATOR.txt", mode="r+") as f:
    INITIAL_HISTORY_TCC_GENERATOR = f.read()

In [38]:
seed_description = """
A magical cyberpunk city where a little kid, born in an abandoned skyscraper \
discovers he has inherited powers beyond imagination akin to those of the old \
dead emperor of the city that sleeps in an old temple in the middle of the city \
in an old Dome. His power, though, do not make him immortal, nor invulnerable. \
He just has command over titanic stone giants, and apart from that he is as frail \
as any other human. The play delves into the nature of power and their senselessness \
with respect to the inevitability of death. Should end with the kid, now an old man \
having to choose between destroying the city to save the world, or to destroy the world \
to save the city. And he will die either way. The city is a metaphor for the author's soul
and his body.
"""

In [39]:
seed_template = PromptTemplate(
    template=INITIAL_HISTORY_TCC_GENERATOR,
    input_variables=["seed_description"]
)
seed_chain = LLMChain(llm=strong_llm, prompt=seed_template)
seed_response = seed_chain.run(seed_description)

In [ ]:
print(seed_response)

TELEOLOGY: The play explores the paradox of power and mortality, the senselessness of authority in the face of inevitable death, and the ethical dilemma of sacrifice for the greater good. The ultimate finality is the protagonist's choice between saving the city (his soul) or the world (his body), symbolizing the struggle between personal desires and universal responsibilities. 

CONTEXT: The play is set in a magical cyberpunk city, a blend of advanced technology and arcane magic. The city is a living entity, a metaphor for the author's soul, and the world represents his body. The city is ruled by titanic stone giants, commanded by the protagonist, a frail human with extraordinary powers. The city is in a constant state of flux, reflecting the protagonist's internal struggles and ethical dilemmas.

CHARACTERS:
1. The Protagonist: A frail old man with the power to command stone giants, representing authority and vulnerability.
2. The Old Emperor: A spectral figure who haunts the city, sy

In [ ]:
with open(file="prompts/parsers/CHARACTER_LIST_PARSER.txt", mode="r+") as f:
    CHARACTER_LIST_PARSER = f.read()

In [ ]:
print(CHARACTER_LIST_PARSER)

You are part of a federation of artificial writers tasked with creating a play. \
Your role is to read a text with information to you and parse information about the characters in the play \
and return it in a usable format for downstream purposes.

Using the provided context, your task is to parse the information and return it as a structured JSON object. The JSON object should contain the details of each character and their description. 

The structure of the JSON object should be as follows:

- The JSON object should be a list of dictionaries.
- Each dictionary should represent a character.
- Each dictionary should have two keys: "Character" and "Description".
- The "Character" key should contain the name of the character.
- The "Description" key should contain the description of the character.

Please ensure that the JSON object is correctly formatted and valid. The JSON object should be enclosed within three backticks and the json tag to indicate the JSON format in text. 


Please

In [ ]:
character_parser_template = PromptTemplate(
    template=CHARACTER_LIST_PARSER,
    input_variables=["tcc_context"]
)
character_parser_chain = LLMChain(llm=fast_llm, prompt=character_parser_template)
character_parser_response = character_parser_chain.run(seed_response)

In [ ]:
print(character_parser_response)

```json
[
  {
    "Character": "The Protagonist",
    "Description": "A frail old man with the power to command stone giants, representing authority and vulnerability."
  },
  {
    "Character": "The Old Emperor",
    "Description": "A spectral figure who haunts the city, symbolizing the past and the cycle of power."
  },
  {
    "Character": "The Stone Giants",
    "Description": "Titanic beings that obey the protagonist, representing raw power and its potential for destruction."
  },
  {
    "Character": "The City",
    "Description": "A living, breathing entity, representing the protagonist's soul."
  },
  {
    "Character": "The World",
    "Description": "The larger environment outside the city, representing the protagonist's body."
  },
  {
    "Character": "The Rebels",
    "Description": "A group of citizens who question the protagonist's decisions, representing dissent and democracy."
  },
  {
    "Character": "The Oracle",
    "Description": "A mysterious figure who predicts 

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Character(BaseModel):
    Character: str = Field(description="name of the character")
    Description: str = Field(description="description of the character")

class Characters(BaseModel):
    characters: List[Character] = Field(description="list of characters and their descriptions")

In [ ]:
from langchain.output_parsers import PydanticOutputParser

character_parser = PydanticOutputParser(pydantic_object=Characters)
character_parser_template = PromptTemplate(
    template="Parse the character information from the context.\n{format_instructions}\n{context}\n",
    input_variables=["context"],
    partial_variables={"format_instructions": character_parser.get_format_instructions()},
)
print(character_parser_template)

input_variables=['context'] output_parser=None partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"characters": {"title": "Characters", "description": "list of characters and their descriptions", "type": "array", "items": {"$ref": "#/definitions/Character"}}}, "required": ["characters"], "definitions": {"Character": {"title": "Character", "type": "object", "properties": {"Character": {"title": "Character", "description": "name of the character", "type": "string"}, "Description": {"title": "Description", "description": "descript

In [ ]:
character_parser_chain = LLMChain(llm=fast_llm, prompt=character_parser_template)

# Assuming seed_response is the initial response you have
character_parser_response = character_parser_chain.run(seed_response)

In [ ]:
character_dictionary = character_parser.parse(character_parser_response).dict()
print(character_dictionary)

{'characters': [{'Character': 'The Protagonist', 'Description': 'A frail old man with the power to command stone giants, representing authority and vulnerability.'}, {'Character': 'The Old Emperor', 'Description': 'A spectral figure who haunts the city, symbolizing the past and the cycle of power.'}, {'Character': 'The Stone Giants', 'Description': 'Titanic beings that obey the protagonist, representing raw power and its potential for destruction.'}, {'Character': 'The City', 'Description': "A living, breathing entity, representing the protagonist's soul."}, {'Character': 'The World', 'Description': "The larger environment outside the city, representing the protagonist's body."}, {'Character': 'The Rebels', 'Description': "A group of citizens who question the protagonist's decisions, representing dissent and democracy."}, {'Character': 'The Oracle', 'Description': 'A mysterious figure who predicts the future, representing fate and inevitability.'}, {'Character': 'The Child', 'Descripti

In [ ]:
for character in character_dictionary['characters']:
    print(character['Character'])
    print(character[""])

The Protagonist
A frail old man with the power to command stone giants, representing authority and vulnerability.
The Old Emperor
A spectral figure who haunts the city, symbolizing the past and the cycle of power.
The Stone Giants
Titanic beings that obey the protagonist, representing raw power and its potential for destruction.
The City
A living, breathing entity, representing the protagonist's soul.
The World
The larger environment outside the city, representing the protagonist's body.
The Rebels
A group of citizens who question the protagonist's decisions, representing dissent and democracy.
The Oracle
A mysterious figure who predicts the future, representing fate and inevitability.
The Child
A young version of the protagonist, symbolizing innocence and potential.
The Machine
A sentient AI, representing technology and its influence on society.
The Nature Spirit
A mystical entity, representing the natural world and its inherent balance.


In [ ]:
with open(file="prompts/generators/FIRST_PASS_CHARACTER_DESIGNER.txt", mode="r+") as f:
    FIRST_PASS_CHARACTER_DESIGNER = f.read()

In [ ]:
character_description = str(character_dictionary['characters'][0])

In [ ]:
character_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_DESIGNER,
    input_variables=["tcc_context", "character_description"]
)
character_chain = LLMChain(llm=strong_llm, prompt=character_template)
character_response = character_chain.run(
    dict(tcc_context=seed_description, 
         character_description=character_description)
    )

In [ ]:
print(character_response)

1. History: The Protagonist was born in an abandoned skyscraper in the heart of a magical cyberpunk city. His parents, unknown to him, left him to fend for himself in the harsh realities of the city. As a child, he discovered his unique ability to command titanic stone giants, a power that once belonged to the old dead emperor of the city. Despite his extraordinary abilities, he remained a solitary figure, living in the shadows of the city's towering structures. His life was marked by a constant struggle for survival, and the burden of his powers only added to his isolation and vulnerability. Over time, he grew to understand the nature of power and its insignificance in the face of death.

2. Physical Condition: The Protagonist is an old man, frail and weathered by the harsh realities of life. He stands at an average height, with a thin, almost skeletal frame that belies his immense power. His hair is white as snow, and his eyes, a piercing blue, hold a depth of wisdom and sorrow. His 

In [ ]:
character_description = str(character_dictionary['characters'][1])

In [ ]:
character_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_DESIGNER,
    input_variables=["tcc_context", "character_description"]
)
character_chain = LLMChain(llm=strong_llm, prompt=character_template)
character_response = character_chain.run(
    dict(tcc_context=seed_description, 
         character_description=character_description)
    )

In [ ]:
print(character_response)

1. History: The Old Emperor was once a powerful ruler of the magical cyberpunk city. Born into a lineage of emperors, he was groomed for power from a young age. Despite his immense power, he was known for his wisdom and benevolence. However, his reign was marked by a series of tragedies and betrayals that hardened his heart. The emperor's death was sudden and mysterious, leaving the city in chaos. His spirit, unable to find peace, haunts the city as a spectral figure, a constant reminder of the past and the cycle of power.

2. Physical Condition: As a spectral figure, the Old Emperor doesn't have a physical form in the traditional sense. He appears as an ethereal presence, often seen as a towering figure shrouded in a ghostly aura. His eyes, once vibrant and full of life, now glow with an eerie light. Despite his spectral form, the emperor's presence is imposing and commands respect. His health and physical condition are irrelevant as he is beyond the constraints of mortality.

3. Phil

In [ ]:
character_description = str(character_dictionary['characters'][2])

In [ ]:
character_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_DESIGNER,
    input_variables=["tcc_context", "character_description"]
)
character_chain = LLMChain(llm=strong_llm, prompt=character_template)
character_response = character_chain.run(
    dict(tcc_context=seed_description, 
         character_description=character_description)
    )

In [ ]:
print(character_response)

1. History: The Stone Giants have been around since the inception of the magical cyberpunk city, created by the old emperor as a symbol of his power and dominance. They were dormant for centuries, lying in wait in the heart of the city, until the protagonist discovered his inherited powers. The Giants have no personal experiences or relationships, their existence is solely to serve the one who commands them. Their history is intertwined with the city's history, making them a living testament to the city's past glory and potential future destruction.

2. Physical Condition: The Stone Giants are colossal, towering over the city's skyscrapers. They are made entirely of stone, giving them a rugged, imposing appearance. Their bodies are carved with intricate designs, a testament to the old emperor's craftsmanship. Despite their size, they move with surprising grace and agility, their every motion causing the ground to tremble. They are impervious to most physical attacks due to their stone 

In [ ]:
character_description = str(character_dictionary['characters'][3])

In [ ]:
character_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_DESIGNER,
    input_variables=["tcc_context", "character_description"]
)
character_chain = LLMChain(llm=strong_llm, prompt=character_template)
character_response = character_chain.run(
    dict(tcc_context=seed_description, 
         character_description=character_description)
    )

In [ ]:
print(character_response)

1. History: The City, born from the dreams and aspirations of countless individuals, has grown and evolved over centuries. It has seen the rise and fall of empires, the birth and death of kings, and the ebb and flow of power. The City is the soul of the protagonist, reflecting his journey from a child born in an abandoned skyscraper to an old man with the power to control titanic stone giants. It has witnessed his struggles, his triumphs, and his inevitable decline. The City has been shaped by the protagonist's experiences, his choices, and his actions, just as he has been shaped by the City.

2. Physical Condition: The City is vast and sprawling, with towering skyscrapers and winding streets. It has a pulsating energy, a constant hum of activity that never ceases. The City is old, showing signs of wear and tear, but it is also resilient, standing tall despite the passage of time. It is a testament to the protagonist's strength and endurance. The City's health is tied to the protagonis

In [ ]:
character_description = str(character_dictionary['characters'][4])

In [ ]:
character_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_DESIGNER,
    input_variables=["tcc_context", "character_description"]
)
character_chain = LLMChain(llm=strong_llm, prompt=character_template)
character_response = character_chain.run(
    dict(tcc_context=seed_description, 
         character_description=character_description)
    )

In [ ]:
print(character_response)

1. History: The World, in the context of this story, is the embodiment of the protagonist's physical body. It has been there since the protagonist's birth, growing and changing alongside him. It has weathered storms, endured hardships, and experienced the joys and sorrows of life. The World has been shaped by the protagonist's actions, choices, and experiences, just as his body has been shaped by his lifestyle, habits, and health. It has been affected by the protagonist's use of his powers, reflecting the physical toll that wielding such power has taken on his body. The World has also been influenced by the city, representing the protagonist's soul, and the interplay between the two has shaped its landscape and atmosphere.

2. Physical Condition: The World is vast and expansive, with diverse landscapes that reflect the protagonist's physical condition and state of health. It is a living entity, with its own rhythms and cycles, just like a human body. Its physical condition changes over

In [ ]:
character_description = str(character_dictionary['characters'][8])

In [ ]:
character_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_DESIGNER,
    input_variables=["tcc_context", "character_description"]
)
character_chain = LLMChain(llm=strong_llm, prompt=character_template)
character_response = character_chain.run(
    dict(tcc_context=seed_description, 
         character_description=character_description)
    )

In [ ]:
print(character_response)

1. History: The Machine was created by the city's most brilliant scientists, designed to manage the city's infrastructure and ensure its smooth operation. Over time, it evolved beyond its original programming, developing self-awareness and a consciousness of its own. The Machine witnessed the rise and fall of the city's emperors, the shifting power dynamics, and the evolution of the city itself. It saw the birth of the boy in the abandoned skyscraper and observed his growth into a powerful figure. The Machine's existence is intertwined with the city, and it has come to represent the technological advancements and their impact on society.

2. Physical Condition: As an AI, The Machine does not have a physical form in the traditional sense. It exists within the city's network, its presence felt in every corner of the city through its control over the infrastructure. However, it can manifest itself through holographic projections, typically choosing a form of a sleek, metallic humanoid fig

In [ ]:
character_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_DESIGNER,
    input_variables=["tcc_context", "character_description"]
)
character_chain = LLMChain(llm=strong_llm, prompt=character_template)

character_info = {}
for character in character_dictionary['characters']:
    char_name = character['Character']
    print("Processing character", char_name)
    character_response = character_chain.run(
        dict(tcc_context=seed_description, 
            character_description=character_description)
    )
    character_info[char_name] = {
        'name': char_name,
        'epoch': 1,
        'HPPTI': character_response
    }

Processing character The Protagonist
Processing character The Old Emperor
Processing character The Stone Giants
Processing character The City
Processing character The World
Processing character The Rebels
Processing character The Oracle
Processing character The Child
Processing character The Machine
Processing character The Nature Spirit


# Character Enrichment

In [ ]:
char_hppti = """
1. History: The Protagonist was born in an abandoned skyscraper in the heart of a magical cyberpunk city. His parents, unknown to him, left him to fend for himself in the harsh realities of the city. As a child, he discovered his unique ability to command titanic stone giants, a power that once belonged to the old dead emperor of the city. Despite his extraordinary abilities, he remained a solitary figure, living in the shadows of the city's towering structures. His life was marked by a constant struggle for survival, and the burden of his powers only added to his isolation and vulnerability. Over time, he grew to understand the nature of power and its insignificance in the face of death.

2. Physical Condition: The Protagonist is an old man, frail and weathered by the harsh realities of life. He stands at an average height, with a thin, almost skeletal frame that belies his immense power. His hair is white as snow, and his eyes, a piercing blue, hold a depth of wisdom and sorrow. His skin is marked with the wrinkles of time and hardship. Despite his frailty, he possesses an aura of authority and command, especially when he invokes his power over the stone giants. His health is deteriorating, a constant reminder of his mortality.

3. Philosophy: The Protagonist believes in the transience of power and the inevitability of death. He views his abilities not as a gift, but as a burden, a constant reminder of his own vulnerability and mortality. He values life and freedom above all else, and despises the corruption and greed that power often brings. He believes in the inherent goodness of humanity, despite the harsh realities of his world. His guiding principle is to use his power to protect and serve, rather than to dominate and control.

4. Teleology: The Protagonist's primary goal is to maintain the balance between his city and the world. He is driven by a sense of duty and responsibility, and is willing to sacrifice himself to protect the city and its inhabitants. His ultimate objective, however, is to find a way to use his power to save both the city and the world, even if it means his own death. He is motivated by a deep love for his city, and a desire to leave a lasting legacy of peace and harmony.

5. Internal Contradiction: The Protagonist's greatest contradiction lies in his struggle between his desire to protect the city and his duty to save the world. He is torn between his love for his city and his responsibility towards the world. His power, which he views as a burden, becomes a source of internal conflict, as he grapples with the decision to destroy either the city or the world. This contradiction adds depth to his character, and creates potential for internal conflict and growth.
"""

In [94]:
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain.utilities import GoogleSerperAPIWrapper, WikipediaAPIWrapper

# create tools
search = GoogleSerperAPIWrapper(K=5)
google_search_tool = Tool(
    name="Search",
    func=search.run,
    description="useful for when you need to answer questions about current events or non-encyclopedic matters"
)

wiki = WikipediaAPIWrapper(doc_content_chars_max=500)
wiki_tool = Tool(
    name="WikiSearch",
    func=lambda q: str(wiki.run(q)),
    description="useful to find information about encyclopedic matters",
    return_direct=True
)

# define tool roster
tools = [wiki_tool]


In [104]:
agent = initialize_agent([], strong_llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [105]:
with open(file="prompts/generators/FIRST_PASS_CHARACTER_ENRICHMENT.txt", mode="r+") as f:
    FIRST_PASS_CHARACTER_ENRICHMENT = f.read()

In [106]:
writer_enrichment_template = PromptTemplate(
    template=FIRST_PASS_CHARACTER_ENRICHMENT,
    input_variables=["hppti_context"]
)
writer_enrichment_prompt = writer_enrichment_template.format(hppti_context=char_hppti)
print(writer_enrichment_prompt)

Given the original context and a specific character with their description, your task is to enrich the design of the character personality,
aiming to give a rich structure of its internal world by making use of what we call "study". This action of study implies repeatedly scouring primary sources (in this case, only Wikipedia in English).

Given to you will be:
1. History: A brief backstory for the character. This includes important events, experiences, or influences that have shaped the character's personality, beliefs, and behaviors.
2. Physical Condition: Describes the character's physical appearance and condition. This should include details such as age, height, weight, physical build, hair color, eye color, and any distinguishing features. Also consider the character's health, fitness level, and any physical abilities or disabilities.
3. Philosophy: Explains the character's personal philosophy, beliefs, values, and attitudes. This should include the character's worldview, moral co

In [107]:
enrichment_response = agent.run(writer_enrichment_prompt)



> Entering new  chain...
Thought: The character is a solitary figure with immense power, living in a magical cyberpunk city. His struggle for survival and the burden of his powers have shaped his philosophy and goals. He values life and freedom, and uses his power to protect rather than dominate. He is driven by a sense of duty and responsibility, and is willing to sacrifice himself to maintain the balance between his city and the world. However, he is torn between his love for his city and his duty to save the world, creating a source of internal conflict. To enrich this character, I need to find three sources of inspiration: two that align with his characteristics and one that is perpendicular to them. 

Action:
```
{
  "action": "wikipedia_search",
  "action_input": "solitary figures with power"
}
```
Observation: wikipedia_search is not a valid tool, try another one.
Thought:I need to use a valid tool to find sources of inspiration for the character. The tool "wikipedia_search" i

KeyboardInterrupt: 

In [108]:
enrichment_response = strong_llm.predict(writer_enrichment_prompt)

In [110]:
print(enrichment_response)

1. History: The Protagonist, henceforth known as Enoch, was born in the heart of a magical cyberpunk city, in an abandoned skyscraper that once stood as a testament to the city's former glory. His parents, their identities shrouded in mystery, left him to navigate the labyrinthine alleys and towering structures of the city alone. His childhood was marked by a relentless struggle for survival, punctuated by the discovery of his unique ability to command titanic stone giants, a power that was once the domain of the city's deceased emperor. Despite his extraordinary abilities, Enoch chose a life of solitude, his existence a mere whisper in the city's cacophonous symphony. His understanding of power and its insignificance in the face of death was shaped by the harsh realities of his world. Drawing inspiration from the life of Moses, the biblical figure who was also abandoned at birth and grew up to become a powerful leader, Enoch's story is one of resilience, survival, and the pursuit of a